# Compare Matrix Factorization with SVD and SVD++
Also perform hyperparameter tuning for SVD

## load package

In [6]:
import random
import numpy as np

from surprise import SVD, SVDpp, NormalPredictor
from surprise.model_selection import GridSearchCV

from utils.MovieLens import MovieLens
from utils.Evaluator import Evaluator

## utils function

In [2]:
def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

In [ ]:
np.random.seed(0)
random.seed(0)

## read data

In [3]:
# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


## compare SVD with SVD++

In [4]:
# SVD
SVD = SVD()
evaluator.AddAlgorithm(SVD, "SVD")

# SVD++
SVDPlusPlus = SVDpp()
evaluator.AddAlgorithm(SVDPlusPlus, "SVD++")

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

# Fight!
evaluator.Evaluate(False)

evaluator.SampleTopNRecs(ml)

Evaluating  SVD ...
Evaluating accuracy...
Analysis complete.
Evaluating  SVD++ ...
Evaluating accuracy...
Analysis complete.
Evaluating  Random ...
Evaluating accuracy...
Analysis complete.


Algorithm  RMSE       MAE       
SVD        0.9039     0.6984    
SVD++      0.8943     0.6887    
Random     1.4359     1.1493    

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.

Using recommender  SVD

Building recommendation model...
Computing recommendations...

We recommend:
Gladiator (1992) 4.520884890007874
Philadelphia Story, The (1940) 4.420701711947352
Stand by Me (1986) 4.3959589752178365
Moon (2009) 4.372613693384055
Happiness (1998) 4.369493252705134
American Graffiti (1973) 4.353470600109924
And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001) 4.3491924926304995
Wallace & Gromit: A Close Shave (1995) 4.3154412154304085
Band of Brothers (2001) 4.315414828016616
Seven Samurai (Shichinin no

## hyperparameter tuning for SVD

In [7]:
print("Searching for best parameters...")
param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.010],
              'n_factors': [50, 100]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(evaluationData)

# best RMSE score
print("Best RMSE score attained: ", gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Searching for best parameters...
Best RMSE score attained:  0.8998197388777108
{'n_epochs': 20, 'lr_all': 0.005, 'n_factors': 50}


In [8]:
# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

params = gs.best_params['rmse']
SVDtuned = SVD(n_epochs = params['n_epochs'], lr_all = params['lr_all'], n_factors = params['n_factors'])
evaluator.AddAlgorithm(SVDtuned, "SVD - Tuned")

SVDUntuned = SVD()
evaluator.AddAlgorithm(SVDUntuned, "SVD - Untuned")

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

# Fight!
evaluator.Evaluate(False)

evaluator.SampleTopNRecs(ml)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD ...
Evaluating accuracy...
Analysis complete.
Evaluating  SVD++ ...
Evaluating accuracy...
Analysis complete.
Evaluating  Random ...
Evaluating accuracy...
Analysis complete.
Evaluating  SVD - Tuned ...
Evaluating accuracy...
Analysis complete.
Evaluating  SVD - Untuned ...
Evaluating accuracy...
Analysis complete.
Evaluating  Random ...
Evaluating accuracy...
Analysis complete.


Algorithm  RMSE       MAE       
SVD        0.9048     0.7010    
SVD++      0.8970     0.6900    
Random     1.4427     1.1512    
SVD - Tuned 0.9019     0.6979    
SVD - Untuned 0.9030     0.6993    

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.

Using recommender  SVD

Building recommendation model...
Computing recommendations...

We recommend:
African Queen, The (1951) 4.7708545168353